In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go


import warnings
warnings.filterwarnings('ignore')
import gc

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from lightgbm import  LGBMClassifier,early_stopping,log_evaluation


In [ ]:
train_data = pd.read_feather('/kaggle/input/amexfeather/train_data.ftr')

#train_data.head()

In [ ]:
train_data.shape

In [ ]:
#train_data.describe(include='all',datetime_is_numeric=True).T

In [ ]:
#print(train_data['customer_ID'].nunique())
#print(train_data['S_2'].min())
#print(train_data['S_2'].max())

In [ ]:
print("No. of features for each category:")

for perf in ['D_','S_','P_','B_','R_']:
    print(f"{perf}: {len([i for i in train_data.columns if i.startswith(perf)])}")

In [ ]:
#train_data.isnull().sum()     

In [ ]:
#sns.set()
#sns.countplot(x=train_data['target'])

In [ ]:
df_temp = pd.DataFrame(train_data.target.value_counts() * 100 / train_data.shape[0])
df_temp



In [ ]:
#values = df_temp['target'].values
#labels = ['Default','Non Default']

#layout = go.Layout(barmode = 'group') 
#fig = go.Figure(data=[go.Bar(y = values,x=labels)],layout=layout)
#fig.show()


In [ ]:
#percentage of missing values for each feature

#missing_per = train_data.isnull().sum()*100/len(train_data)
#df_missing = pd.DataFrame({'Column': train_data.columns,'percent of missing':missing_per}).sort_values('percent of missing',ascending=False)
#df_missing

In [ ]:

# Keep the last statement month per customer
# https://www.kaggle.com/competitions/amex-default-prediction/discussion/327094
train_data =  (train_data
            .groupby('customer_ID')
            .tail(1)
            .set_index('customer_ID', drop=True)
            .sort_index()
            .drop(['S_2'], axis='columns'))

In [ ]:
_ = gc.collect

In [ ]:
columns_all = train_data.columns.to_list()
columns_cat = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [ ]:
columns_num = [column for column in columns_all if column not in columns_cat + ['target']]

In [ ]:
X = train_data[columns_cat + columns_num]
y= train_data['target']

In [ ]:
#X.head()

In [ ]:
encoder = OrdinalEncoder()
X[columns_cat] = encoder.fit_transform(X[columns_cat])

In [ ]:
%time
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,stratify=y)

#stratify parameter makes a split so that the proportion of values in the sample produced will be the same as the proportion of values provided to parameter stratify.
#For example, if variable y is a binary categorical variable with values 0 and 1 and there are 25% of zeros and 75% of ones, 
#stratify=y will make sure that your random split has 25% of 0's and 75% of 1's.

In [ ]:
%time
model = LGBMClassifier(n_estimators=50000,device='gpu',random_state=69420,extra_trees=True)

In [ ]:
model.fit(
            X_train,
            y_train,
            eval_set=[(X_test,y_test)],
            callbacks=[early_stopping(25),log_evaluation(0)]
        )

In [ ]:
y_predict = pd.DataFrame(y_test.copy(deep=True))
y_predict=y_predict.rename(columns={'target':'prediction'})
y_predict

In [ ]:
y_predict["prediction"] = model.predict_proba(X_test)[:,1]
y_predict

# Submission

In [ ]:
_ = gc.collect()

In [ ]:
test_data = pd.read_feather('../input/amexfeather/test_data.ftr')
test_data.head(5)

In [ ]:
test_data = (test_data.groupby('customer_ID').tail(1)).drop(['S_2'],axis='columns')
test_data

In [ ]:
test_data[columns_cat] = encoder.transform(test_data[columns_cat])
test_data

In [ ]:
test_data['prediction'] = model.predict_proba(test_data[columns_cat + columns_num])[:,1]
test_data.head(10)

In [ ]:
test_data['prediction'].to_csv('submission.csv',index=False)

In [ ]:
#kaggle competitions submit -c amex-default-prediction -f submission.csv -m "Message"